# Homework 6: Mixed effects

This homework assignment is designed to give you practice fitting and interpreting mixed effects models. 

We will be using the **LexicalData.csv** and **Items.csv** files from the *Homework/lexDat* folder in the class GitHub repository again. 

This data is a subset of the [English Lexicon Project database](https://elexicon.wustl.edu/). It provides the reaction times (in milliseconds) of many subjects as they are presented with letter strings and asked to decide, as quickly and as accurately as possible, whether the letter string is a word or not. The **Items.csv** provides characteristics of the words used, namely frequency (how common is this word?) and length (how many letters?). Unlike in the previous homework, there isn't any missing data in the **LexicalData.csv** file. 

*Data courtesy of Balota, D.A., Yap, M.J., Cortese, M.J., Hutchison, K.A., Kessler, B., Loftis, B., Neely, J.H., Nelson, D.L., Simpson, G.B., & Treiman, R. (2007). The English Lexicon Project. Behavior Research Methods, 39, 445-459.*

---
## 1. Loading and formatting the data (1 point)

Load in data from the **LexicalData.csv** and **Items.csv** files. As in the previous homeworks, remove the commas from the reaction times and convert them from strings to numbers. Use `left_join` to add word characteristics `Length` and `Log_Freq_Hal` from **Items** to **LexicalData**. 

*Note: the `Freq_HAL` variable in **Items.csv** has a similar formatting issue, using string values with commas. We're not going to worry about fixing this since we're only using `Log_Freq_HAL`, which is the natural log transformation of `Freq_HAL`, in this homework.*

In [31]:
library(dplyr)
setwd("/Users/asalyunusova/Documents/Data_Science/Homework/lexDat") #load lexical data
lexDat <- read.csv("LexicalData.csv")
lexDat$D_RT<- gsub(",", "", lexDat$D_RT) #remove commas from reaction time
lexDat$D_RT=as.numeric(lexDat$D_RT) #convert reaction time from strings to numbers
head(lexDat)
setwd("/Users/asalyunusova/Documents/Data_Science/Homework/lexDat") #load items data
ItemsDat <- read.csv("Items.csv")
head(ItemsDat)
lexDat_renamed <- rename(lexDat, 
                          Word = D_Word) #match 'word' columns to execute left_join
names(lexDat_renamed)
lexitems <-
  left_join(lexDat_renamed,
            ItemsDat %>% dplyr::select(Word, Length, Log_Freq_HAL),
            by = "Word") #add characteristics length and log_freq_HAL
head(lexitems)

,Sub_ID,Trial,Type,D_RT,D_Word,Outlier,D_Zscore
,<int>,<int>,<int>,<dbl>,<chr>,<chr>,<dbl>
1,157,1,1,710,browse,false,-0.437
2,67,1,1,1094,refrigerant,false,0.825
3,120,1,1,587,gaining,false,-0.645
4,21,1,1,984,cheerless,false,0.025
5,236,1,1,577,pattered,false,-0.763
6,236,2,1,715,conjures,false,-0.364


,Occurrences,Word,Length,Freq_HAL,Log_Freq_HAL
,<int>,<chr>,<int>,<chr>,<dbl>
1,1,synergistic,11,284,5.649
2,1,synonymous,10,951,6.858
3,1,syntactical,11,114,4.736
4,1,synthesis,9,"6,742",8.816
5,1,synthesized,11,"2,709",7.904
6,1,synthesizer,11,"1,390",7.237


[1] "Sub_ID"   "Trial"    "Type"     "D_RT"     "Word"     "Outlier"  "D_Zscore"

,Sub_ID,Trial,Type,D_RT,Word,Outlier,D_Zscore,Length,Log_Freq_HAL
,<int>,<int>,<int>,<dbl>,<chr>,<chr>,<dbl>,<int>,<dbl>
1,157,1,1,710,browse,false,-0.437,6,8.856
2,67,1,1,1094,refrigerant,false,0.825,11,4.644
3,120,1,1,587,gaining,false,-0.645,7,8.304
4,21,1,1,984,cheerless,false,0.025,9,2.639
5,236,1,1,577,pattered,false,-0.763,8,1.386
6,236,2,1,715,conjures,false,-0.364,8,5.268


---
## 2. Model fitting (4 points)

First, fit a linear model with `Log_Freq_HAL` and `Length` as predictors, and `D_RT` as the output. Include an interaction term. Use `summary()` to look at the model output. 

In [33]:
lm.fit = lm(D_RT~Log_Freq_HAL+Length+Log_Freq_HAL:Length, data = lexitems) #Create the linear model with an interaction term
summary(lm.fit) #Review the results


Call:
lm(formula = D_RT ~ Log_Freq_HAL + Length + Log_Freq_HAL:Length, 
    data = lexitems)

Residuals:
     Min       1Q   Median       3Q      Max 
-1118.01  -205.23   -86.95    90.77  3147.07 

Coefficients:
                    Estimate Std. Error t value Pr(>|t|)    
(Intercept)         610.1903    14.6678  41.601  < 2e-16 ***
Log_Freq_HAL         -6.0239     1.9678  -3.061  0.00221 ** 
Length               47.7531     1.6368  29.175  < 2e-16 ***
Log_Freq_HAL:Length  -2.9421     0.2348 -12.528  < 2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 359.1 on 62606 degrees of freedom
Multiple R-squared:  0.09473,	Adjusted R-squared:  0.09469 
F-statistic:  2184 on 3 and 62606 DF,  p-value: < 2.2e-16


Now, install `lme4` using `install.packages()` and then load the library. 

In [18]:
# Install LME4
install.packages("lme4")
library(lme4)


The downloaded binary packages are in
	/var/folders/_b/t7fr6fg53h970lfthzdnkjnr0000gp/T//RtmpTtgMkY/downloaded_packages


Now fit a mixed effects model that includes the same predictors as the linear model above, as well as random intercepts for `Sub_ID` (i.e., cases where subject ID shifts the RT mean). Use `summary()` to look at the model output. 

In [34]:
me.fit1 = lmer(D_RT ~ Log_Freq_HAL + Length + Log_Freq_HAL:Length + (1 | Sub_ID), data=lexitems) # mixed model with subject random intercepts
summary(me.fit1) 

Linear mixed model fit by REML ['lmerMod']
Formula: D_RT ~ Log_Freq_HAL + Length + Log_Freq_HAL:Length + (1 | Sub_ID)
   Data: lexitems

REML criterion at convergence: 888235.6

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-4.5058 -0.5472 -0.1568  0.3103 10.7381 

Random effects:
 Groups   Name        Variance Std.Dev.
 Sub_ID   (Intercept) 46333    215.3   
 Residual             82978    288.1   
Number of obs: 62610, groups:  Sub_ID, 299

Fixed effects:
                    Estimate Std. Error t value
(Intercept)         616.8445    17.1522  35.963
Log_Freq_HAL         -7.4374     1.5830  -4.698
Length               47.7477     1.3162  36.277
Log_Freq_HAL:Length  -2.8778     0.1888 -15.239

Correlation of Fixed Effects:
            (Intr) Lg_F_HAL Length
Log_Frq_HAL -0.645                
Length      -0.656  0.917         
Lg_Fr_HAL:L  0.582 -0.942   -0.923

---
## 3. Model assessment (4 points)

Compare the three t-values for the fixed effects and the mixed effects models. How do they differ, and why? 

> *The fixed effect t-value for Log_Freq_HAL has decreased to -4.698, from -3.061 in the linear model. The fixed effect t-value for length has increased to 36.277, from 29.175 in the simple linear model. The fixed effect t-value for the interaction term Log_Freq_HAL:Length has decreased to -15.239, from -12.528 in the simple linear model. The  t values differ because we get a more conservative linear model fit compared to the fixed effects fit for Log_Freq_HAL and Log_Freq_HAL:Length.*

>*The mixed effects model is telling us about subject differences in reaction times compared to the linear model. Overall, the mixed effect model is telling us that the effect of Length, Log_Freq_HAL, and Length:Log_Freq_HAL on reaction time varies by subject which is why there is a difference in t-values between the two models.*

>*The t-value for random effects are not reported because as stated in the lecture "interpreting the magnitude of the random effect is meaningless because we defined those effects as tangential (irrelevant) to the fixed effect that we are looking for".*

Use the Aikeke Information Criterion (AIC) to compare these two models. Which one is better? 

In [44]:
# compare the two models using the Akaike information criterion (AIC)
ic = AIC(lm.fit, me.fit1)
ic
diff(ic$AIC)

,df,AIC
,<dbl>,<dbl>
lm.fit,5,914436.4
me.fit1,6,888247.6


[1] -26188.82

> *The mixed effects model has the lowest AIC, therefore it is the best model to choose for the data. The difference in AIC between lm.fit and me.fit1 is very large (~25960.5), this is telling us that the mixed effect model is accounting for more variance in reaction time than the simple linear model, even after accounting for subject ID.*

---
##  4. Reflection (1 point)

What other random effects could be controlled for in this data set? 

> *The variable "Word" because the type of word appearing to subjects is random, therefore the effect of Length, Log_Freq_HAL, and Length:Log_Freq_HAL on reaction time might vary by Word. We can also use a boxplot and scatterplot to see if specific words have different reaction times or reveal any patterns of clustering (e.g., the words that start with 'a' are grouped together in faster reaction times*
> 

**DUE:** 5pm EST, March 25, 2022

**IMPORTANT** Did you collaborate with anyone on this assignment? If so, list their names here. 
> *Jenah Black, Julia Conti, Emefa Akwayena*